# Finance Agent using LangChain (LlaMa-3 | Groq API)

## Table of Contents

1. **Load LLM (LlaMa-3)**
2. **Define Finance Tools**
3. **Create Finance Agent (LLM + Tools)**

## Finance Agent Capabilities

* **Stock Company Basic Information**
    * Address, Industry, Sector, business summary, website, etc.
    * Top Company Officers
* **Stock Financials**
    * Market Cap, current price, ebitda, total debt, total revenue, debt-to-equity, P/E ratio, operating margins, ebitda margins, operating cashflow, free cashflow, etc.
* **Stock Dividend & Earnings Dates**
* **Information about Mutual Fund Holders**
* **Information about Institutional Holders**
* **Stock Splits History**
* **Stock Ratings Upgrades & Downgrades by various Companies**
* **News Articles about Stock**

In [ ]:
import dotenv

dotenv.load_dotenv('/Users/ali.shamsaddinlou/Documents/codes/moonky/.env')

In [ ]:
import os
print(os.getcwd())

In [ ]:
import os

groq_api_key = os.environ["GQ_API_KEYRO"]

In [ ]:
import yfinance as yf

yf.__version__

## 1. Load LLM

* Login to **https://console.groq.com** and create API Key.

### Groq Models

ID|	REQUESTS PER MINUTE|	REQUESTS PER DAY|	TOKENS PER MINUTE
-|-|-|-
llama3-70b-8192	|30	|14,400	|6,000
llama3-8b-8192	|30	|14,400	|30,000
gemma-7b-it	|30	|14,400	|15,000
mixtral-8x7b-32768	|30	|14,400	|5,000


* **pip install langchain-groq**

In [ ]:
from langchain_groq import ChatGroq

llama3 = ChatGroq(api_key="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", model="llama-3.1-70b-versatile")

llama3

In [ ]:
llama3.invoke("Hello, How are you?")

## 2. Define Tools

In [ ]:
from langchain_core.tools import tool, StructuredTool
from datetime import date

@tool
def company_information(ticker: str) -> dict:
    """Use this tool to retrieve company information like address, industry, sector, company officers, business summary, website,
       marketCap, current price, ebitda, total debt, total revenue, debt-to-equity, etc."""
    
    ticker_obj = yf.Ticker(ticker)
    ticker_info = ticker_obj.get_info()

    return ticker_info

@tool
def last_dividend_and_earnings_date(ticker: str) -> dict:
    """
    Use this tool to retrieve company's last dividend date and earnings release dates.
    It does not provide information about historical dividend yields.
    """
    ticker_obj = yf.Ticker(ticker)
    
    return ticker_obj.get_calendar()

@tool
def summary_of_mutual_fund_holders(ticker: str) -> dict:
    """
    Use this tool to retrieve company's top mutual fund holders. 
    It also returns their percentage of share, stock count and value of holdings.
    """
    ticker_obj = yf.Ticker(ticker)
    mf_holders = ticker_obj.get_mutualfund_holders()
    
    return mf_holders.to_dict(orient="records")

@tool
def summary_of_institutional_holders(ticker: str) -> dict:
    """
    Use this tool to retrieve company's top institutional holders. 
    It also returns their percentage of share, stock count and value of holdings.
    """
    ticker_obj = yf.Ticker(ticker)   
    inst_holders = ticker_obj.get_institutional_holders()
    
    return inst_holders.to_dict(orient="records")

@tool
def stock_grade_updrages_downgrades(ticker: str) -> dict:
    """
    Use this to retrieve grade ratings upgrades and downgrades details of particular stock.
    It'll provide name of firms along with 'To Grade' and 'From Grade' details. Grade date is also provided.
    """
    ticker_obj = yf.Ticker(ticker)
    
    curr_year = date.today().year
    
    upgrades_downgrades = ticker_obj.get_upgrades_downgrades()
    upgrades_downgrades = upgrades_downgrades.loc[upgrades_downgrades.index > f"{curr_year}-01-01"]
    upgrades_downgrades = upgrades_downgrades[upgrades_downgrades["Action"].isin(["up", "down"])]
    
    return upgrades_downgrades.to_dict(orient="records")

@tool
def stock_splits_history(ticker: str) -> dict:
    """
    Use this tool to retrieve company's historical stock splits data.
    """
    ticker_obj = yf.Ticker(ticker)
    hist_splits = ticker_obj.get_splits()
    
    return hist_splits.to_dict()

@tool
def stock_news(ticker: str) -> dict:
    """
    Use this to retrieve latest news articles discussing particular stock ticker.
    """
    ticker_obj = yf.Ticker(ticker)
    
    return ticker_obj.get_news()


#stock_news_tool = StructuredTool.from_function(stock_news)

## 3.  Define Agent (LLM + Tools)

In [ ]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder

tools = [
         company_information,
         last_dividend_and_earnings_date,
         stock_splits_history,
         summary_of_mutual_fund_holders,
         summary_of_institutional_holders, 
         stock_grade_updrages_downgrades,
         stock_news
]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Try to answer user query using available tools.",
        ),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llama3 = ChatGroq(api_key=groq_api_key, model="llama-3.1-70b-versatile", temperature=0)

finance_agent = create_tool_calling_agent(llama3, tools, prompt)

finance_agent_executor = AgentExecutor(agent=finance_agent, tools=tools, verbose=True)

In [ ]:
from langchain_core.messages import HumanMessage

resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is address of microsoft?")]})

In [ ]:
print(resp["output"])

In [ ]:
#finance_agent_executor.verbose = False

In [ ]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is last close price of microsoft?")]})

In [ ]:
print(resp["output"])

In [ ]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="What is dividend yield for microsoft stock?")]})

In [ ]:
print(resp["output"])

In [ ]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="What are ebitda, total debt, total revenue and debt-to-equity for Nvidia stock?")]})

In [ ]:
print(resp["output"])

In [ ]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="Summarize Key people at apple")]})

In [ ]:
print(resp["output"])

In [ ]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="What was last dividend date for apple stock?")]})

In [ ]:
print(resp["output"])

In [ ]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="Top mutual fund holders of microsoft stock.")]})

In [ ]:
print(resp["output"])

In [ ]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="Top institutional holders of microsoft stock.")]})

In [ ]:
print(resp["output"])

In [ ]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="Which firms recently changed grade of apple stock? Also, return rating changes.")]})

In [ ]:
print(resp["output"])

In [ ]:
resp = finance_agent_executor.invoke({"messages": [HumanMessage(content="Can you share latest news articles links discussing apple stocks?")]})

In [ ]:
print(resp["output"])

## Summary

In this video, I explained how to create **finance agent** using **LangChain**. We used open source LLM **llama-3** for our purpose.